In [0]:
#all libraries used
!apt install chromium-chromedriver
!pip install selenium 
!pip install fuzzywuzzy
import math
import random
import urllib.request
import requests
import ssl
import time
from googlesearch import search
import re
from bs4 import BeautifulSoup
!pip install --upgrade firebase-admin
from fuzzywuzzy import fuzz
from selenium import webdriver
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


Initialized empty Git repository in /content/.git/

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@5cf57c0c8e4b.(none)')
error: src refspec master does not match any.
error: failed to push some refs to 'https://github.com/azuzow/oogaBooga.git'


In [0]:
#gets all webpages for query
pagesWithoutRankings=[]
amzonLinks=[]
def getLinks(query='top 10 mirrorless cameras'):
  domainList = []
  searchList= []
  for count,i in enumerate (search(query, tld="com", num=40, start=0, stop=40, pause=2)):
      domain = i.split('//')[-1].split('/')[0]
      domain = domain.lstrip('www.')
      domain = "https://www." + domain
      #if a domain has already been seen dont add the link to the pages that will be scraped
      if domain not in domainList:
          domainList.append(domain)
          print(count)
          # print(i)
          searchList.append(i)
  # print(searchList)
  return searchList
def visible(element):
  if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
    return False
  elif re.match('<!--.*-->', str(element.encode('utf-8'))):
    return False
  return True

def getPages(urls):
  pages = []
  cc=1
  amazonLinks=set()
  #problems occur here, i think when try doesnt work the first time? im not sure
  junkWords=["These ","quite","to","too","like","This",'this','because',"it's"]
  # junkWords=[]

  for items in urls:
      try:
          html = requests.get(items,timeout=5).text
          soup = BeautifulSoup(html,'html.parser')

          data = soup.findAll(text=True)

          #finds all links then gets all links with amazon referenced 
          links = [a.get('href') for a in soup.find_all('a', href=True)]
          for link in links:
            if 'amazon' in link:
              if(link not in amazonLinks):
                amazonLinks.add(link)
                # print(link)
     
          visible_texts = filter(visible, data)
          temp=''
          for t in visible_texts:
            junkLine=False
            for w in junkWords: 
              if w in t:
                junkLine=True
                break
            if not junkLine:
              # print("==============================")
              # print(t)
              # print(t.strip())
              # print(type(t))
              temp =temp+ (t.strip()+"\n")
              # print(temp)
              # print("==============================")
          visible_texts=temp
          pages.append(visible_texts)
      except Exception as e:
          print('couldnt get page',items)
          print(e)
      #soup = BeautifulSoup(html)
      #data = soup.findAll(text=True)
      print(cc)
      cc+=1
  return pages,amazonLinks

  #how many pages have visible text to scrape 
  # print(len(pages))
#all pages with visible text from query
def findLists():
  pages,amazonLinks=getPages(getLinks())

  lists=[]

  regEx = "\d+[.][' '][a-zA-z].*"

  for page in pages:
    #come back and do something with page if no rankings are found
    lists.append(re.findall(regEx,page))
  # print("=================================== PRODUCT NAMES ===============================")
  # for list in lists:
  #   print(list)
  # print("=================================== PRODUCT NAMES ===============================")
  return lists,amazonLinks





In [0]:
 def listformat(lstoflst):
  #this section turns each item into a list [rank,name], so we have [[[rank, name][rank,name]][[rank,name][rank,name]]]
  newlist=[]
  for i in range(len(lstoflst)):
    templist=[]
    print("============================= PAGE",i+1,"=============================")
    for k in range(len(lstoflst[i])):
      print(lstoflst[i][k])
      rank=lstoflst[i][k][0:3]
      name=lstoflst[i][k][3:]
      templist.append([rank,name])
    newlist.append(templist)
  return(newlist)

      



def listcleanblanks(lst):
  for i in (lst[:]):
    if i==[]:
      lst.remove(i)
  return(lst)

def removespaceandcombo(lst):
  for i in range(len(lst)):
    for k in range(len(lst[i])):
      
      if lst[i][k][1][0] == ' ':
        lst[i][k][1]=lst[i][k][1][1:]
      if lst[i][k][0][-1]==' ':
        lst[i][k][0]=lst[i][k][0][:-1]
  newlist=[]
  for i in range(len(lst)):
    for k in range(len(lst[i])):
      newlist.append(lst[i][k])

  return(newlist)

def getAmazonASIN(amazonLinks):
  productsASIN=set()
  urls=set()
  for link in amazonLinks:
      try:
        headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, sdch, br",
        "Accept-Language": "en-US,en;q=0.8",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"}
        regEx= '[A-Z0-9]{10}[?]'
        request = requests.get(link, headers=headers, allow_redirects=True)
        url = request.url
        ASIN = re.search(regEx,url)
        if(ASIN!=None):
          urls.add(url)
          ASIN = ASIN.group(0)        
          ASIN=ASIN[:10]
          productsASIN.add(ASIN)
      except Exception as e:
        print(e) 
  return productsASIN    


lst,amazonLinks=findLists()
lst=listformat(lst)
lst=listcleanblanks(lst)
#lst=removespaceandcombo(lst)
amazonASIN= getAmazonASIN(amazonLinks)
print(*lst,sep='/n')
print('-----------------------------')
numoflists=len(lst)



0
2
3
7
8
16
17
18
19
20
21
22
23
24
25
27
28
29
30
31
32
33
34
35
36
37
38
39
1
2
3
4
5
6
7
8
9
10
11
12
couldnt get page https://www.pcmag.com/roundup/297663/the-best-dslr-and-mirrorless-cameras?sa=X&ved=2ahUKEwjy8onKn_7mAhVKUlAKHUI5BoIQFjAOegQIHRAB
HTTPSConnectionPool(host='www.pcmag.com', port=443): Read timed out. (read timeout=5)
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
============================= PAGE 1 =============================
============================= PAGE 2 =============================
1. Nikon Z50
2. Fujifilm X-T30
3. Olympus OM-D E-M5 Mark III
4. Fujifilm X-T3
5. Nikon Z 6
6. Canon EOS RP
7. Panasonic Lumix GX80/GX85
8. Sony A6000
9. Olympus OM-D E-M10 Mark III
10. Sony A7R Mark IV
11. Nikon Z 7
12. Panasonic Lumix S1R
13. Sony A9 (the A9 II has been announced)
14. Olympus OM-D E-M1 X
15. Fujifilm GFX 50R
============================= PAGE 3 =============================
1. Nikon Z6
2. Sony Alpha A7 III
3. Fujifilm X-T30
4. Sony A7R IV
5. Panasonic Lumix 

In [0]:
def createASINPairs(ASIN):
  trueNames=dict()
  wd = webdriver.Chrome('chromedriver',options=options)
  for id in ASIN:
    try:
      query = 'http://www.amazon.com/s?k=ASIN&ref=nb_sb_noss'
      query= query.replace('ASIN',str(id))
      wd.get(query)
      trueNames[wd.find_element_by_class_name('s-image').get_attribute("alt")]=id
    except Exception as e:
      print(e,query)
  for k, v in trueNames.items():
    print (k, '-->', v)
  return trueNames
ooga=createASINPairs(amazonASIN)



Message: no such element: Unable to locate element: {"method":"css selector","selector":".s-image"}
  (Session info: headless chrome=79.0.3945.79)
 http://www.amazon.com/s?k=B01MQEK9YJ&ref=nb_sb_noss
Message: no such element: Unable to locate element: {"method":"css selector","selector":".s-image"}
  (Session info: headless chrome=79.0.3945.79)
 http://www.amazon.com/s?k=B00U6SBUIM&ref=nb_sb_noss
Message: no such element: Unable to locate element: {"method":"css selector","selector":".s-image"}
  (Session info: headless chrome=79.0.3945.79)
 http://www.amazon.com/s?k=B07WSRSS8K&ref=nb_sb_noss
Message: no such element: Unable to locate element: {"method":"css selector","selector":".s-image"}
  (Session info: headless chrome=79.0.3945.79)
 http://www.amazon.com/s?k=B07S7FNG4S&ref=nb_sb_noss
Message: no such element: Unable to locate element: {"method":"css selector","selector":".s-image"}
  (Session info: headless chrome=79.0.3945.79)
 http://www.amazon.com/s?k=B079X2QJ46&ref=nb_sb_noss


In [0]:
def similar(product, url):
  Token_Set_Ratio = fuzz.token_set_ratio(product.lower(),url.lower())
  #print(Token_Set_Ratio)
  return Token_Set_Ratio

def compareNames(lst,nameDict):
  rankings=[]
  for site in lst:
    for i in range(len(site)):
      for name in nameDict:
        if similar(site[i][1],name) >= 80:
          rankings.append([site[i][1],similar(site[i][1],name),name])
 # print(*rankings,sep='\n')
  rankings = sorted(rankings, key = lambda x: (x[0], x[1]))
  rankings.reverse()
  print(*rankings,sep='\n')
  newlist=[]
  for site in lst:
    for i in range(len(site)):
      for k in range(len(rankings)):
        if site[i][1] == rankings[k][0]:
          newlist.append([site[i][0],rankings[k][0],rankings[k][2]])
          break;
  #print(*newlist,sep='\n')
  return newlist
def connectasin(lst,namedict):
  newlist=[]
  for item in lst:
    newlist.append([item[0],item[2],namedict[item[2]]])
  print(newlist)
  return newlist
one=compareNames(lst,ooga)
two=connectasin(one,ooga)  
print(*two,sep='\n')
# lst=compareNames(lst,ooga)
# lst=connectasin(lst,ooga)

['Sony a7R IV', 84, 'Sony a7R III Mirrorless Camera: 42.4MP Full Frame High Resolution Mirrorless Interchangeable Lens Digital Camera with Front End LSI Image Processor, 4K HDR Video and 3" LCD Screen - ILCE7RM3/B Body']
['Sony Alpha a6300', 100, 'Sony Alpha a6300 Mirrorless Camera: Interchangeable Lens Digital Camera with APS-C, Auto Focus & 4K Video - ILCE 6300 Body with 3” LCD Screen - E Mount Compatible - Black (Includes Body Only)']
['Sony Alpha a6300', 100, 'Sony Alpha a6300 Mirrorless Camera Interchangeable Lens Digital Camera with APS-C, Auto Focus & 4K Video - ILCE 6300/S Body with 3” LCD Screen - E Mount Compatible - Silver (Includes Body Only)']
['Sony Alpha a6000', 100, 'Sony Alpha a6000 Mirrorless Digital Camera 24.3MP SLR Camera with 3.0-Inch LCD (Black) w/16-50mm Power Zoom Lens']
['Sony Alpha A9', 87, 'Sony Alpha a6300 Mirrorless Camera: Interchangeable Lens Digital Camera with APS-C, Auto Focus & 4K Video - ILCE 6300 Body with 3” LCD Screen - E Mount Compatible - Black

In [0]:
#recieving list of lists, each element of the form ['rank from a site', 'name', 'ASIN']
def organizetriplelist(lst):
  for i in range(len(lst)):
    loc=lst[i][0].find('.')
    lst[i][0]=lst[i][0][0:loc]
  for i in range(len(lst)):
    lst[i].reverse()
    lst[i][-1]=int(lst[i][-1])
    
  return lst



def listcompression(lst):
  asin=[]
  newlist=[]
  for i in range(len(lst)):
    asin.append(lst[i][0])
  asin=set(asin)
  for i in asin:
     newlist.append([i])
  for i in range(len(newlist)):
    for k in range(len(lst)):
      if lst[k][0]==newlist[i][0]:
        if len(newlist[i])==1:
          newlist[i].append(lst[k][1])
        newlist[i].append(lst[k][2])

  for i in range(len(newlist)):
    newlist[i].append((sum(newlist[i][2:]))/(len(newlist[i])-2))
    newlist[i].append(len(newlist[i])-3)
  return(newlist)

def addnumoflists(lst,num):
  for i in range(len(lst)):
    lst[i].append(num)
  return lst


 #five = [asin,name,...,ave,freq,#oflists]
def addscore(lst):
  for i in range(len(lst)):
    score = lst[i][-3] + -20*(math.log((2/3)+((lst[i][-2])/(lst[i][-1]))))
    lst[i].append(score)
  return lst


def finalsortbyscore(lst):
  lst.sort(key = lambda lst: lst[-1])
  return lst

print('===========================================')
three=organizetriplelist(two)
# print(*three,sep='\n')
# print('===========================================')
four=listcompression(three)
# print(*four,sep='\n')
# print('===========================================')
five=addnumoflists(four,numoflists)
# print(*five,sep='\n')
# print('===========================================')
six=addscore(five)
#print(*six,sep='\n')

seven = finalsortbyscore(six)
print('===========================================')
print(*seven,sep='\n')






# lst=testing()
# print('========================')
# lst=listcompression(lst)
# print(*lst,sep='\n')

['B01M050N05', 'PANASONIC Lumix G85 4K Digital Camera, 12-60mm Power O.I.S. Lens, 16 Megapixel Mirrorless Camera, 5 Axis In-Body Dual Image Stabilization, 3-Inch Tilt and Touch LCD, DMC-G85MK (Black)', 7, 9, 16, 3, 3, 3, 7, 3, 6.375, 8, 12, 0.6213585509643833]
['B07NSBN5JF', 'Fujifilm X-T30 Mirrorless Digital Camera (Body Only, Black) w/Deluxe Bundle', 2, 3, 5, 6, 4, 6, 4.333333333333333, 6, 12, 1.2503197367881698]
['B07N9KDCGV', 'Canon EOS RP Mirrorless Camera Body, Black - 3380C002', 6, 6, 2, 5, 9, 6, 9, 6.142857142857143, 7, 12, 1.6799861165729482]
['B07GPRSYG8', 'Nikon Z6 Full Frame Mirrorless Camera Body', 1, 4, 11, 1, 4, 8, 4.833333333333333, 6, 12, 1.7503197367881698]
['B07H3Y9CJY', 'Fujifilm X-T3 Mirrorless Digital Camera (Body Only) - Black', 4, 1, 2, 2, 10, 3.8, 5, 12, 2.1991458465292726]
['B01NCVN74T', 'Fujifilm X-T20 Mirrorless Digital Camera w/XF18-55mmF2.8-4.0 R LM OIS Lens-Silver', 9, 14, 3, 2, 4, 6, 9, 6.714285714285714, 7, 12, 2.2514146880015193]
['B0751B835N', 'Olympu